# Testing downloads from APPLAUSE

We ned to download two kinds of data: plate scans, and source tables. Analysis requires two plates and four source tables:

- plate scan 1 
- plate scan 2
- sources 1
- sources calib 1
- sources 2
- sources calib 2

In [1]:
import os
import shutil
from importlib import reload
from urllib.parse import urlsplit
import requests
import json

import pyvo as vo

from astropy.io import fits
from astropy.utils.data import download_file
from astropy.table import Table

from settings import DATAPATH, images

## Functions

In [2]:
def download_scan(table, plate_id, test=False):

    # get URL for this plate ID 
    mask = table['plate_id'] == plate_id
    table_1 = table[mask]
    url_1 = table_1['filename_scan'][0]
    
    # get file name from URL
    parsed_url = urlsplit(url_1)
    filename = parsed_url.path.split('/')[-1]

    # Download the file and get the local filename/path
    local_file_path = download_file(url_1)
    if test:
        print(f"File downloaded to: {local_file_path}")

    # Move and rename file into data directory
    new_path = shutil.move(local_file_path, os.path.join(DATAPATH, filename))
    if test:
        print("new path is: ", new_path)

    # TEST: open the local FITS file using the obtained path
    if test:
        with fits.open(new_path) as hdul:
            # Access the header information or data
            header_info = hdul[0].header
            print(f"TEST: telescope used: {header_info.get('TELESCOP')}")
            print(f"TEST: date: {header_info.get('DATE-AVG')}")

    # once image is successfully downloaded, we need to update the 'images.json' 
    # dictionary with the new association "plate id: file name" entry.
    images[str(plate_id)] = filename        
            
    try:
        json_file = open('images.json', 'w')
        json.dump(images, json_file, indent=4)
    except IOError as e:
        print(f"Error writing to file: {e}")

In [3]:
def download_sources_table(plate_id, calib=False):
    # setup
    name = 'APPLAUSE',
    url = 'https://www.plate-archive.org/tap'
    token = 'Token 2e2225623fdf3e89aef67a04415ade1d9369193e'

    # assemble query string
    qstr = 'SELECT * FROM applause_dr4.source'
    if calib:
        qstr = qstr + '_calib'
    qstr = qstr + ' WHERE plate_id = ' + str(plate_id) + ' ORDER BY source_id'

#     print('\npyvo version %s \n' % vo.__version__)
#     print('TAP service %s \n' % name)

    tap_session = requests.Session()
    tap_session.headers['Authorization'] = token

    tap_service = vo.dal.TAPService(url, session=tap_session)

    lang = 'PostgreSQL'

    job = tap_service.submit_job(qstr, language=lang)
    job.run()

    job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=30.)

    job.raise_if_error()
    results = job.fetch_result()
    
    return results.to_table()

## Downloading plate scans

In [4]:
catalog_applause = Table.read('footprints_5.csv', format='ascii.csv')

plate_id = 9535

In [5]:
download_scan(catalog_applause, plate_id)

File downloaded to: /var/folders/y5/zcwj7zc17g19nggvb1vx4rc80000gn/T/astropy-download-66497-4fn_uucp
new path is:  /Users/busko/Projects/VASCO_data/footprints/GS00994_x.fits


## Downloading source tables

In [6]:
results = download_sources_table(9320)

results

source_id,process_id,scan_id,plate_id,archive_id,source_num,x_source,y_source,a_source,b_source,theta_source,erra_source,errb_source,errtheta_source,elongation,x_peak,y_peak,flag_usepsf,x_image,y_image,erra_image,errb_image,errtheta_image,x_psf,y_psf,erra_psf,errb_psf,errtheta_psf,mag_auto,magerr_auto,flux_auto,fluxerr_auto,mag_iso,magerr_iso,flux_iso,fluxerr_iso,flux_max,flux_radius,isoarea,sqrt_isoarea,background,sextractor_flags,dist_center,dist_edge,annular_bin,flag_rim,flag_negradius,flag_clean,model_prediction,timestamp_insert,timestamp_update
,,,,,,,,pix,pix,deg,pix,pix,deg,,,,,,,pix,pix,deg,,,pix,pix,deg,mag,mag,,,mag,mag,,,,pix,pix2,pix,,,pix,pix,,,,,,Time[yyyy-mm-dd hh:mm:ss],Time[yyyy-mm-dd hh:mm:ss]
int64,int32,int32,int32,int32,int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,int32,int32,int16,float64,float64,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,int16,float64,float64,int16,int16,int16,int16,float32,str32,str32
40349400000001,34940,12211,9320,102,1,9084.494,0.2943855308175838,5.1837416,0.28867507,0.0,0.3564568,0.019802745,0.0,17.957012,9084,3,0,9084.494,3.0,0.3564568,0.019802745,0.0,--,--,--,--,--,9.33811,0.017775944,1839737.9,30113.33,12.600807,0.07446627,91133.3,6248.9478,5094.0566,9.723231,18,4.2426405,-3406.477,16,5441.247,3.0,9,1,0,0,0.0018999279,2022-06-13 05:05:14.965387+00,2022-06-13 05:05:14.965387+00
40349400000002,34940,12211,9320,102,2,9348.236,0.5794665765192413,14.547974,0.4431754,-0.32608882,0.4716521,0.014375291,-0.33106926,32.82667,9343,3,0,9348.236,3.2837903,0.4716521,0.014375291,-0.33106926,--,--,--,--,--,8.017025,0.010520553,6211408.0,60172.605,10.974314,0.035097085,407637.75,13173.938,5196.098,20.685041,80,8.944272,-4097.495,16,5609.449,3.2837903,9,1,0,0,0.0018999279,2022-06-13 05:05:14.965387+00,2022-06-13 05:05:14.965387+00
40349400000003,34940,12211,9320,102,3,9473.502,1.0235610362455194,16.115112,1.1469431,0.34483317,0.31037036,0.022153009,0.35426667,14.050489,9478,3,0,9473.502,3.7258742,0.31037036,0.022153009,0.35426667,--,--,--,--,--,7.4671035,0.008661055,10307627.0,82205.27,9.842738,0.020432608,1155859.1,21746.977,5722.422,28.663033,218,14.764823,-4075.8384,16,5691.6475,3.7258742,9,1,0,0,0.0018999279,2022-06-13 05:05:14.965387+00,2022-06-13 05:05:14.965387+00
40349400000004,34940,12211,9320,102,4,11041.712,17.95166559288308,1.0288509,0.47537577,-8.464025,0.110559426,0.050700705,-8.350907,2.1642897,11041,21,0,11041.712,20.577337,0.110559426,0.050700705,-8.350907,--,--,--,--,--,11.827339,0.07253796,185807.95,12410.801,12.958821,0.064577006,65534.742,3896.9036,10022.97,2.5217178,7,2.6457512,1.0259863,0,6825.255,20.577337,9,1,0,0,5.885723e-30,2022-06-13 05:05:14.965387+00,2022-06-13 05:05:14.965387+00
40349400000005,34940,12211,9320,102,5,11058.012,17.086123111654548,4.7944274,0.61618704,-2.907899,0.20365995,0.026140172,-2.9401805,7.780799,11055,20,0,11058.012,19.715714,0.20365995,0.026140172,-2.9401805,--,--,--,--,--,10.427391,0.038453385,674596.0,23886.28,11.197873,0.029325841,331780.5,8959.247,10793.856,7.2559285,37,6.0827627,1.0632463,19,6838.581,19.715714,9,1,0,0,3.4629274e-24,2022-06-13 05:05:14.965387+00,2022-06-13 05:05:14.965387+00
40349400000006,34940,12211,9320,102,6,10973.496,20.38534674507381,2.6748285,0.6528963,-0.33533254,0.15936054,0.038877536,0.015136459,4.096866,10973,23,0,10973.496,23.0,0.15936054,0.038877536,0.015136459,--,--,--,--,--,11.623888,0.07553787,224101.48,15587.614,11.848555,0.037244193,182212.48,6248.9478,12592.974,2.54361,18,4.2426405,1.023085,3,6770.364,23.0,9,1,0,0,3.5283199e-31,2022-06-13 05:05:14.965387+00,2022-06-13 05:05:14.965387+00
40349400000007,34940,12211,9320,102,7,10963.506,21.389894805786618,3.243181,0.5855648,-1.0548021,0.19808406,0.03558724,-0.9519638,5.538552,10964,24,0,10963.506,24.0,0.19808406,0.03558724,-0.9519638,--,--,--,--,--,10.458835,0.038592618,655339.3,23288.453,11.79624,0.03741